In [1]:
%matplotlib notebook
from matplotlib import pyplot as plt
import matplotlib as mpl

In [2]:
from tqdm import tqdm

In [3]:
from datetime import datetime

In [4]:
import json

In [5]:
import pandas as pd

In [6]:
from pymongo import MongoClient
client = MongoClient()
db = client['hkns3']

In [ ]:
df_times = pd.DataFrame(list(db['raw'].find({"type":"max_id"})))

In [ ]:
df_times['datetime'] = pd.to_datetime(df_times['time']*10**9)

In [ ]:
df_times

In [ ]:
df_times.plot(x='datetime', y='test')

In [ ]:
df_times.set_value(0, 'test', 0)

In [ ]:
def derivative_time():
    prev_id = 14349758
    prev_time = 1494944438.3075223
    for row in df_times.itertuples():
        index = row[0]
        if index > 0:
            time_diff = (row[3] - prev_time)
            id_dff = row[2] - prev_id
            df_times.set_value(index, 'time_diff',(time_diff*1.0))
            df_times.set_value(index, 'test',(id_dff/time_diff*1.0))
            prev_id = row[2]
            prev_time = row[3]

In [ ]:
derivative_time()

In [ ]:
df_times

In [ ]:
df_times['time'][0]

In [ ]:
df_times['time_diff'] = pd.Series()

In [ ]:
bs = list(db['raw'].find({"type":"best"}))

In [ ]:
len(bs[0]['bests'])

In [ ]:
df_bs = pd.DataFrame(columns=['_id', 'time', 'rank'])

In [ ]:
df_bs.append({'_id': 1, 'time': 0, 'rank': 0}, ignore_index=True)

In [ ]:
df_bs

In [ ]:
with open("bests.csv", "w") as b:
    b.write("i,item_id,time,rank\n")
    c = 0
    for entry in tqdm(bs):
        time = entry['time']
        for i, v in enumerate(entry['bests']):
            rank = i + 1
            _id = v
            b.write(f"{c},{_id},{time},{rank}\n")
            c += 1

In [ ]:
df_bs = pd.read_csv("bests.csv", index_col=0)

In [ ]:
df_bs[df_bs['item_id'] == 14326439]

In [ ]:
df_bs.groupby('item_id').size().sort_values()

In [ ]:
df_bs['datetime'] = pd.to_datetime(df_bs['time']*10**9)

In [ ]:
df_bs[df_bs['item_id'] == 14330547].plot(x='datetime', y='rank')

In [ ]:
df_bs

Now let do for **top**

In [ ]:
ts = list(db['raw'].find({"type":"top"}))
with open("tops.csv", "a") as tf:
#    tf.write("i,item_id,time,rank\n")
    c = 2355047
    for entry in tqdm(ts):
        time = entry['time']
        for i, v in enumerate(entry['bests']):
            rank = i + 1
            _id = v
            tf.write(f"{c},{_id},{time},{rank}\n")
            c += 1

In [7]:
df_ts = pd.read_csv("tops.csv", index_col=0)

In [ ]:
df_ts['datetime'] = pd.to_datetime(df_ts['time']*10**9)

In [ ]:
plt.gca().invert_yaxis()

In [ ]:
df_ts.groupby('item_id').size().sort_values()

In [ ]:
df_ts_ts2 = df_ts[df_ts['item_id'] == 14218408][['time', 'rank']].set_index('time')

In [8]:
df_ts_unique = df_ts['item_id'].unique()

In [ ]:
json.dumps(df_ts_ts2.to_dict())

In [ ]:
df_ts_ts2 = df_ts[df_ts['item_id'] == 14218408][['time', 'rank']]

In [ ]:
df_ts_ts2['ntime'] = ((df_ts_ts2['time']*10**5).astype(int))

In [ ]:
bb = [(str(x[0]), int(x[1])) for x in df_ts_ts2[['ntime', 'rank']].values]

In [ ]:
json.dumps({"rankings": bb})

In [ ]:
str(df_ts_ts2[['ntime', 'rank']].values[1][0])

In [ ]:
bb

In [13]:
def get_time_series(_id):
    dfs = df_ts[df_ts['item_id'] == _id][['time', 'rank']]
    dfs['ntime'] = ((dfs['time']*10**5).astype(int))
    bb = [(str(x[0]), int(x[1])) for x in dfs[['ntime', 'rank']].values]
    return {
        "id": _id,
        "rankings": bb
    }

In [14]:
def insert_timeseries():
    col = db['time_series']
    for _id in tqdm(df_ts_unique):
        _id = int(_id)
        ts = get_time_series(_id)
        col.insert_one(ts)

In [15]:
insert_timeseries()


100%|██████████| 3556/3556 [02:46<00:00, 25.08it/s]
